<a href="https://colab.research.google.com/github/ilham-mukti/Machine-Learning/blob/main/data_teks/Document_Similarity_Data_Tirto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

import numpy as np
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

sw_indo = stopwords.words("indonesian") + list(punctuation)

In [4]:
datas = pd.read_csv("/content/data_tirto.csv")
datas.head()

,id_berita,judul_berita,isi_berita,nama_penulis,date_created,url_artikel
0,1259889,Jokowi Sindir ASN yang Minta Dilayani: Seperti...,Presiden Jokowi menilai para aparatur sipil ne...,Andrian Pratama Taher,27/07/2021 10:55,https://tirto.id/jokowi-sindir-asn-yang-minta-...
1,1259829,Larangan Demo Jokowi End Game & Bahaya Doxing ...,"Jumat, 23 Juli 2021, sekira pukul 23-an, ada t...",Adi Briantika,26/07/2021 17:24,https://tirto.id/larangan-demo-jokowi-end-game...
2,1259921,"Jokowi: Mahasiswa Punya Talenta, Bisa Berprofe...",Presiden Joko Widodo meminta agar perguruan ti...,Andrian Pratama Taher,27/07/2021 13:36,https://tirto.id/jokowi-mahasiswa-punya-talent...
3,1259902,Tjahjo Minta Seluruh ASN Ikuti Arahan Jokowi-M...,Menteri Pendayagunaan Aparatur Negara dan Refo...,Andrian Pratama Taher,27/07/2021 12:30,https://tirto.id/tjahjo-minta-seluruh-asn-ikut...
4,1259792,"Jokowi, Menkes dan Mendag Disomasi Imbas Kelan...",Koalisi masyarakat sipil menyomasi Presiden Jo...,Irwan Syambudi,26/07/2021 13:41,https://tirto.id/jokowi-menkes-dan-mendag-diso...


In [5]:
datas.shape

(1303, 6)

In [6]:
datas.drop_duplicates(subset='id_berita', inplace=True, ignore_index=True)

In [7]:
datas.shape

(863, 6)

# Ekstrak TFIDF

In [8]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(datas.isi_berita)
tfidf_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<863x243099 sparse matrix of type '<class 'numpy.float64'>'
	with 475069 stored elements in Compressed Sparse Row format>

# Document Similarity

In [32]:
#Kalo pake Distance
def document_similarity(index, matrix, topk=10):
  sim = cosine_distances(matrix[index], matrix)
  sorted_sim = sim.argsort()[0, 0:11]
  berita_yang_sama = [index for index in reversed(sorted_sim[-topk:])]
  return sorted_sim

In [33]:
data_ke = 10
result_index = document_similarity(data_ke, tfidf_matrix)
print(result_index)

[ 10  23  31  97 737 575 159 168 217 121 632]


In [38]:
#Kalo pake Similarity, hasilnya sama aja
def document_similarity(index, matrix, topk=10):
  sim = cosine_similarity(matrix[index], matrix)
  sorted_sim = sim[0].argsort()
  berita_yang_sama = [index for index in reversed(sorted_sim[-topk:])]
  return berita_yang_sama

In [39]:
data_ke = 10
result_index = document_similarity(data_ke, tfidf_matrix)
print(result_index)

[10, 23, 31, 97, 737, 575, 159, 168, 217, 121]


In [34]:
datas.loc[result_index, 'id_berita': 'nama_penulis']

,id_berita,judul_berita,isi_berita,nama_penulis
10,1259966,RS Darurat COVID-19 di Parkiran RS Sardjito Ra...,Menteri Pekerjaan Umum dan Perumahan Rakyat (P...,Selfie Miftahul Jannah
23,1259930,Penambahan Kapasitas RS Lapangan Bantul Ditarg...,Menteri Pekerjaan Umum dan Perumahan Rakyat (P...,Gilang Ramadhan
31,1260029,"Kasus COVID Terus Melonjak, Pemerintah Kebut R...",Menteri Pekerjaan Umum dan Perumahan Rakyat (P...,Selfie Miftahul Jannah
97,1259532,RS Darurat GOR Indoor GBT Surabaya Mulai Berop...,Rumah Sakit Darurat GOR Indoor Komplek Gelora ...,Gilang Ramadhan
737,1258830,Bagaimana Jakarta & DIY Hadapi Skenario Terbur...,Menteri Kesehatan Budi Gunadi Sadikin menyebut...,Irwan Syambudi
575,1259011,Penjelasan Satgas soal Strategi Pemerintah Red...,Pemerintah Indonesia telah memutuskan Pemberla...,Addi M Idhom
159,1260060,Gubernur Kepri Desak Kemenkes Lunasi Biaya Per...,Gubernur Kepulauan Riau Ansar Ahmad mendesak K...,tidak ada
168,1259968,Kasus COVID-19 di DIY Melonjak & Rekor Kematia...,Penambahan kasus baru COVID-19 di Provinsi Dae...,Irwan Syambudi
217,1259823,BOR RS DKI Turun Tipis Jadi 73 Persen & 89 Per...,Tingkat keterisian tempat tidur rumah sakit at...,Riyan Setiawan
121,1259483,LaporCOVID-19: Ada 2.313 Kematian Saat Isoman ...,Koalisi warga untuk LaporCOVID-19 mencatat jum...,Irwan Syambudi


In [35]:
def extract_keyword_tfidf(doc, tfidf, topk=10):
  matrix = tfidf.transform([doc])
  vocab = tfidf.get_feature_names()

  sorted_tfidf = matrix[0].toarray()[0].argsort()
  return([vocab[idx] for idx in reversed(sorted_tfidf[-topk:])])

In [36]:
text = datas.isi_berita[19]
extract_keyword_tfidf(text, tfidf)

['stigmatisasi',
 'isoman',
 'kematian',
 'covid',
 'vino',
 'budi',
 'sulfikar',
 'menkes budi',
 'penderita covid',
 'penderita']